In [1]:
!pip install qiskit matplotlib numpy scipy plotly pandas scikit-learn torch gym stable-baselines3
!pip install qiskit qiskit-aer stable-baselines3 matplotlib gymnasium shimmy
!pip install qiskit qiskit-aer gym stable-baselines3 matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of shimmy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 10.4 MB/s eta 0:00:00


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
# Set up the path for saving the model
path = "/content/drive/My Drive/Colab Notebooks/Quantum Realm/models/"
os.makedirs(path, exist_ok=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_state_qsphere
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim


def generate_quantum_state_image(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for _ in range(np.random.randint(1, 5)):
        qubit = np.random.randint(0, num_qubits)
        gate = np.random.choice(['h', 'x', 'y', 'z'])
        if gate == 'h':
            qc.h(qubit)
        elif gate == 'x':
            qc.x(qubit)
        elif gate == 'y':
            qc.y(qubit)
        else:
            qc.z(qubit)

    backend = Aer.get_backend('statevector_simulator')
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc)
    result = job.result()
    statevector = result.get_statevector()

    fig, ax = plt.subplots(figsize=(5, 5))
    plot_state_qsphere(statevector, ax=ax)

    # Save the plot to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)

    # Convert the buffer to an image
    buf.seek(0)
    image = Image.open(buf)
    image = image.convert('RGB')

    # Convert to numpy array
    image_array = np.array(image)

    return image_array, qc.draw(output='text')

# Generate dataset
num_samples = 1000
num_qubits = 3
images = []
labels = []

for _ in range(num_samples):
    image, circuit = generate_quantum_state_image(num_qubits)
    images.append(image)
    labels.append(circuit)

images = np.array(images)
labels = np.array(labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train).permute(0, 3, 1, 2) / 255.0
X_test = torch.FloatTensor(X_test).permute(0, 3, 1, 2) / 255.0


In [ ]:
class QuantumStateClassifier(nn.Module):
    def __init__(self):
        super(QuantumStateClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 125 * 125, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 classes for 4 gates

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 125 * 125)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = QuantumStateClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
model_save_path = os.path.join(path, 'quantum_state_classifier.pth')
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(len(X_test)):
        outputs = model(X_test[i].unsqueeze(0))
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == y_test[i]).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')